In [6]:
import pandas as pd
import glob
import os
import re

# Paste your OpenNeuro subject list here (one per line)
openneuro_subject_list = """
s3://openneuro.org/ds005237/sub-NDARINVAG023WG3
s3://openneuro.org/ds005237/sub-NDARINVAG339WHH
s3://openneuro.org/ds005237/sub-NDARINVAG388HJL
s3://openneuro.org/ds005237/sub-NDARINVAG900RVD
s3://openneuro.org/ds005237/sub-NDARINVAH529JMM
s3://openneuro.org/ds005237/sub-NDARINVAK834VNU
s3://openneuro.org/ds005237/sub-NDARINVAL101MH2
s3://openneuro.org/ds005237/sub-NDARINVAM061NXD
s3://openneuro.org/ds005237/sub-NDARINVAN576KX1
s3://openneuro.org/ds005237/sub-NDARINVAP729WCD
s3://openneuro.org/ds005237/sub-NDARINVAR463UNP
s3://openneuro.org/ds005237/sub-NDARINVAT097DFG
s3://openneuro.org/ds005237/sub-NDARINVAZ218MB7
s3://openneuro.org/ds005237/sub-NDARINVBD216MCC
s3://openneuro.org/ds005237/sub-NDARINVBE389YGF
s3://openneuro.org/ds005237/sub-NDARINVBH217XFZ
s3://openneuro.org/ds005237/sub-NDARINVBH315KUM
s3://openneuro.org/ds005237/sub-NDARINVBL062HTE
s3://openneuro.org/ds005237/sub-NDARINVBL128ZXR
s3://openneuro.org/ds005237/sub-NDARINVBL733HBP
s3://openneuro.org/ds005237/sub-NDARINVBM990HJT
s3://openneuro.org/ds005237/sub-NDARINVBN249GWM
s3://openneuro.org/ds005237/sub-NDARINVBU789GV0
s3://openneuro.org/ds005237/sub-NDARINVBY805EE5
s3://openneuro.org/ds005237/sub-NDARINVBZ622PEX
s3://openneuro.org/ds005237/sub-NDARINVCA042YER
s3://openneuro.org/ds005237/sub-NDARINVCE244AGN
s3://openneuro.org/ds005237/sub-NDARINVCK288RP2
s3://openneuro.org/ds005237/sub-NDARINVCL131GYQ
s3://openneuro.org/ds005237/sub-NDARINVCM621YRY
s3://openneuro.org/ds005237/sub-NDARINVCP169JDZ
s3://openneuro.org/ds005237/sub-NDARINVCV410NUH
s3://openneuro.org/ds005237/sub-NDARINVCW125BLA
s3://openneuro.org/ds005237/sub-NDARINVCW577CWF
s3://openneuro.org/ds005237/sub-NDARINVCX643TCM
s3://openneuro.org/ds005237/sub-NDARINVCX685EVQ
s3://openneuro.org/ds005237/sub-NDARINVDC524THW
s3://openneuro.org/ds005237/sub-NDARINVDD155BRR
s3://openneuro.org/ds005237/sub-NDARINVDG233EBR
s3://openneuro.org/ds005237/sub-NDARINVDK220VPQ
s3://openneuro.org/ds005237/sub-NDARINVDM785DVB
s3://openneuro.org/ds005237/sub-NDARINVDN485GPF
s3://openneuro.org/ds005237/sub-NDARINVDP288XND
s3://openneuro.org/ds005237/sub-NDARINVDT499KZL
s3://openneuro.org/ds005237/sub-NDARINVDU085XVZ
s3://openneuro.org/ds005237/sub-NDARINVDV232BEQ
s3://openneuro.org/ds005237/sub-NDARINVDW733XXB
s3://openneuro.org/ds005237/sub-NDARINVDZ608PPY
s3://openneuro.org/ds005237/sub-NDARINVEA806MCW
s3://openneuro.org/ds005237/sub-NDARINVEC746UWL
s3://openneuro.org/ds005237/sub-NDARINVED812LMR
s3://openneuro.org/ds005237/sub-NDARINVEF266GBV
s3://openneuro.org/ds005237/sub-NDARINVEG178LHD
s3://openneuro.org/ds005237/sub-NDARINVEJ537VGZ
s3://openneuro.org/ds005237/sub-NDARINVEK183ZLE
s3://openneuro.org/ds005237/sub-NDARINVEK685MY0
s3://openneuro.org/ds005237/sub-NDARINVEN039PVQ
s3://openneuro.org/ds005237/sub-NDARINVEP476HJ3
s3://openneuro.org/ds005237/sub-NDARINVEV975LY3
s3://openneuro.org/ds005237/sub-NDARINVEY033HCZ
s3://openneuro.org/ds005237/sub-NDARINVEY681MZ7
s3://openneuro.org/ds005237/sub-NDARINVFE128JJV
s3://openneuro.org/ds005237/sub-NDARINVFH503ZWA
s3://openneuro.org/ds005237/sub-NDARINVFJ172LJ5
s3://openneuro.org/ds005237/sub-NDARINVFT463JPQ
s3://openneuro.org/ds005237/sub-NDARINVFU389BX1
s3://openneuro.org/ds005237/sub-NDARINVFW143KVU
s3://openneuro.org/ds005237/sub-NDARINVFW820XN0
s3://openneuro.org/ds005237/sub-NDARINVFW876XT7
s3://openneuro.org/ds005237/sub-NDARINVFX289XV7
s3://openneuro.org/ds005237/sub-NDARINVFY729CMQ
s3://openneuro.org/ds005237/sub-NDARINVGB107TDU
s3://openneuro.org/ds005237/sub-NDARINVGB371PPV
s3://openneuro.org/ds005237/sub-NDARINVGG504ENV
s3://openneuro.org/ds005237/sub-NDARINVGH969TWR
s3://openneuro.org/ds005237/sub-NDARINVGK662YZW
s3://openneuro.org/ds005237/sub-NDARINVGM287EF8
s3://openneuro.org/ds005237/sub-NDARINVGN063ZRV
s3://openneuro.org/ds005237/sub-NDARINVGN082RP7
s3://openneuro.org/ds005237/sub-NDARINVGR029VCQ
s3://openneuro.org/ds005237/sub-NDARINVGR746CR0
s3://openneuro.org/ds005237/sub-NDARINVGT021UPR
s3://openneuro.org/ds005237/sub-NDARINVGT486MAN
s3://openneuro.org/ds005237/sub-NDARINVGU013UHX
s3://openneuro.org/ds005237/sub-NDARINVGZ602BF8
s3://openneuro.org/ds005237/sub-NDARINVHA329EL1
s3://openneuro.org/ds005237/sub-NDARINVHB925HEK
s3://openneuro.org/ds005237/sub-NDARINVHG032NYJ
s3://openneuro.org/ds005237/sub-NDARINVHT721HFR
s3://openneuro.org/ds005237/sub-NDARINVHV402VH9
s3://openneuro.org/ds005237/sub-NDARINVHW100CDA
s3://openneuro.org/ds005237/sub-NDARINVHY331CLY
s3://openneuro.org/ds005237/sub-NDARINVHZ510TB2
s3://openneuro.org/ds005237/sub-NDARINVJA418ZRD
s3://openneuro.org/ds005237/sub-NDARINVJC140HGJ
s3://openneuro.org/ds005237/sub-NDARINVJK969JZ8
s3://openneuro.org/ds005237/sub-NDARINVJP343BJ6
s3://openneuro.org/ds005237/sub-NDARINVJP751NXV
s3://openneuro.org/ds005237/sub-NDARINVJR138MBQ
s3://openneuro.org/ds005237/sub-NDARINVJT215VYQ
s3://openneuro.org/ds005237/sub-NDARINVJT253NWQ
s3://openneuro.org/ds005237/sub-NDARINVJT886CG5
s3://openneuro.org/ds005237/sub-NDARINVJV338PGX
s3://openneuro.org/ds005237/sub-NDARINVJX155XLR
s3://openneuro.org/ds005237/sub-NDARINVJY908YB9
s3://openneuro.org/ds005237/sub-NDARINVKC627BAV
s3://openneuro.org/ds005237/sub-NDARINVKD900ED7
s3://openneuro.org/ds005237/sub-NDARINVKF855DZG
s3://openneuro.org/ds005237/sub-NDARINVKH279ZDZ
s3://openneuro.org/ds005237/sub-NDARINVKH356XFP
s3://openneuro.org/ds005237/sub-NDARINVKH965NN0
s3://openneuro.org/ds005237/sub-NDARINVKP945BWF
s3://openneuro.org/ds005237/sub-NDARINVKV870NBK
s3://openneuro.org/ds005237/sub-NDARINVKW897YWP
s3://openneuro.org/ds005237/sub-NDARINVKX727WL8
s3://openneuro.org/ds005237/sub-NDARINVKZ112BTB
s3://openneuro.org/ds005237/sub-NDARINVKZ413VTU
s3://openneuro.org/ds005237/sub-NDARINVKZ712GTY
s3://openneuro.org/ds005237/sub-NDARINVLC145NV2
s3://openneuro.org/ds005237/sub-NDARINVLD269XMU
s3://openneuro.org/ds005237/sub-NDARINVLK466LC2
s3://openneuro.org/ds005237/sub-NDARINVLK689LJ2
s3://openneuro.org/ds005237/sub-NDARINVLK739LPV
s3://openneuro.org/ds005237/sub-NDARINVLL260KC0
s3://openneuro.org/ds005237/sub-NDARINVLT949NAG
s3://openneuro.org/ds005237/sub-NDARINVLZ686XNX
s3://openneuro.org/ds005237/sub-NDARINVMH676UAR
s3://openneuro.org/ds005237/sub-NDARINVMJ687EBC
s3://openneuro.org/ds005237/sub-NDARINVMV972LBE
s3://openneuro.org/ds005237/sub-NDARINVMZ631XR9
s3://openneuro.org/ds005237/sub-NDARINVMZ789RWY
s3://openneuro.org/ds005237/sub-NDARINVNB949AXM
s3://openneuro.org/ds005237/sub-NDARINVND653BE6
s3://openneuro.org/ds005237/sub-NDARINVNE865PBN
s3://openneuro.org/ds005237/sub-NDARINVPB396GT2
s3://openneuro.org/ds005237/sub-NDARINVPD575FK1
s3://openneuro.org/ds005237/sub-NDARINVPE293RXE
s3://openneuro.org/ds005237/sub-NDARINVPE364JB5
s3://openneuro.org/ds005237/sub-NDARINVPF283TAQ
s3://openneuro.org/ds005237/sub-NDARINVPF308MTF
s3://openneuro.org/ds005237/sub-NDARINVPF766MJ2
s3://openneuro.org/ds005237/sub-NDARINVPG675JPX
s3://openneuro.org/ds005237/sub-NDARINVPG851GUA
s3://openneuro.org/ds005237/sub-NDARINVPJ213YAX
s3://openneuro.org/ds005237/sub-NDARINVPT961JTN
s3://openneuro.org/ds005237/sub-NDARINVPU175NP8
s3://openneuro.org/ds005237/sub-NDARINVPZ987BMX
s3://openneuro.org/ds005237/sub-NDARINVRB157VE8
s3://openneuro.org/ds005237/sub-NDARINVRB271ZFF
s3://openneuro.org/ds005237/sub-NDARINVRC807HPA
s3://openneuro.org/ds005237/sub-NDARINVRF868XAA
s3://openneuro.org/ds005237/sub-NDARINVRF914JT6
s3://openneuro.org/ds005237/sub-NDARINVRR054KAM
s3://openneuro.org/ds005237/sub-NDARINVRW134NUR
s3://openneuro.org/ds005237/sub-NDARINVRX371YHK
s3://openneuro.org/ds005237/sub-NDARINVRZ105MC1
s3://openneuro.org/ds005237/sub-NDARINVTA573RU2
s3://openneuro.org/ds005237/sub-NDARINVTC494BH2
s3://openneuro.org/ds005237/sub-NDARINVTF281GWR
s3://openneuro.org/ds005237/sub-NDARINVTH522AEV
s3://openneuro.org/ds005237/sub-NDARINVTR059ATR
s3://openneuro.org/ds005237/sub-NDARINVTR903THR
s3://openneuro.org/ds005237/sub-NDARINVTT359WEC
s3://openneuro.org/ds005237/sub-NDARINVTT812VKB
s3://openneuro.org/ds005237/sub-NDARINVTU813PDR
s3://openneuro.org/ds005237/sub-NDARINVTV991YAD
s3://openneuro.org/ds005237/sub-NDARINVUA181LXU
s3://openneuro.org/ds005237/sub-NDARINVUR466KN5
s3://openneuro.org/ds005237/sub-NDARINVUT195DEQ
s3://openneuro.org/ds005237/sub-NDARINVUV598MXY
s3://openneuro.org/ds005237/sub-NDARINVUX111AA3
s3://openneuro.org/ds005237/sub-NDARINVUX642KUY
s3://openneuro.org/ds005237/sub-NDARINVUY536RET
s3://openneuro.org/ds005237/sub-NDARINVUY799LKJ
s3://openneuro.org/ds005237/sub-NDARINVUZ656BRT
s3://openneuro.org/ds005237/sub-NDARINVVC008EZL
s3://openneuro.org/ds005237/sub-NDARINVVD461TM8
s3://openneuro.org/ds005237/sub-NDARINVVF051GV0
s3://openneuro.org/ds005237/sub-NDARINVVH854UEQ
s3://openneuro.org/ds005237/sub-NDARINVVK074AWR
s3://openneuro.org/ds005237/sub-NDARINVVP179WTP
s3://openneuro.org/ds005237/sub-NDARINVVT280VDN
s3://openneuro.org/ds005237/sub-NDARINVVU614ZKP
s3://openneuro.org/ds005237/sub-NDARINVVV366BKJ
s3://openneuro.org/ds005237/sub-NDARINVVZ816AVQ
s3://openneuro.org/ds005237/sub-NDARINVWA310HH4
s3://openneuro.org/ds005237/sub-NDARINVWB541TEM
s3://openneuro.org/ds005237/sub-NDARINVWD109LR7
s3://openneuro.org/ds005237/sub-NDARINVWD338PY2
s3://openneuro.org/ds005237/sub-NDARINVWD467AR0
s3://openneuro.org/ds005237/sub-NDARINVWE937RD6
s3://openneuro.org/ds005237/sub-NDARINVWF881BPQ
s3://openneuro.org/ds005237/sub-NDARINVWG867JHJ
s3://openneuro.org/ds005237/sub-NDARINVWJ708RM0
s3://openneuro.org/ds005237/sub-NDARINVWJ892FLH
s3://openneuro.org/ds005237/sub-NDARINVWL810FRT
s3://openneuro.org/ds005237/sub-NDARINVWM327ZZN
s3://openneuro.org/ds005237/sub-NDARINVWM533NJC
s3://openneuro.org/ds005237/sub-NDARINVWN600UP5
s3://openneuro.org/ds005237/sub-NDARINVWR872ZDB
s3://openneuro.org/ds005237/sub-NDARINVWT248DFY
s3://openneuro.org/ds005237/sub-NDARINVWU297KRB
s3://openneuro.org/ds005237/sub-NDARINVWZ534JVA
s3://openneuro.org/ds005237/sub-NDARINVXA261ZAL
s3://openneuro.org/ds005237/sub-NDARINVXD089VAD
s3://openneuro.org/ds005237/sub-NDARINVXE784YJ5
s3://openneuro.org/ds005237/sub-NDARINVXH435XG7
s3://openneuro.org/ds005237/sub-NDARINVXJ707NAE
s3://openneuro.org/ds005237/sub-NDARINVXM223BAP
s3://openneuro.org/ds005237/sub-NDARINVXP963GZ5
s3://openneuro.org/ds005237/sub-NDARINVXR625UBQ
s3://openneuro.org/ds005237/sub-NDARINVXV404VJL
s3://openneuro.org/ds005237/sub-NDARINVXZ023ZLG
s3://openneuro.org/ds005237/sub-NDARINVXZ387TC1
s3://openneuro.org/ds005237/sub-NDARINVYA281VEM
s3://openneuro.org/ds005237/sub-NDARINVYE059KWM
s3://openneuro.org/ds005237/sub-NDARINVYK619FTF
s3://openneuro.org/ds005237/sub-NDARINVYR744ZAU
s3://openneuro.org/ds005237/sub-NDARINVYT858CBN
s3://openneuro.org/ds005237/sub-NDARINVYZ203GF8
s3://openneuro.org/ds005237/sub-NDARINVZB382MHL
s3://openneuro.org/ds005237/sub-NDARINVZB896FPZ
s3://openneuro.org/ds005237/sub-NDARINVZC713KY8
s3://openneuro.org/ds005237/sub-NDARINVZC781XW2
s3://openneuro.org/ds005237/sub-NDARINVZF221XAB
s3://openneuro.org/ds005237/sub-NDARINVZF290GFY
s3://openneuro.org/ds005237/sub-NDARINVZF426RL4
s3://openneuro.org/ds005237/sub-NDARINVZF605GZ8
s3://openneuro.org/ds005237/sub-NDARINVZH090MNG
s3://openneuro.org/ds005237/sub-NDARINVZK672XPE
s3://openneuro.org/ds005237/sub-NDARINVZL449UYG
s3://openneuro.org/ds005237/sub-NDARINVZR981ALE
s3://openneuro.org/ds005237/sub-NDARINVZT152JCX
s3://openneuro.org/ds005237/sub-NDARINVZU586UPF
s3://openneuro.org/ds005237/sub-NDARINVZU840GFR
s3://openneuro.org/ds005237/sub-NDARINVZV968GA8
s3://openneuro.org/ds005237/sub-NDARINVZW239ZXZ
s3://openneuro.org/ds005237/sub-NDARINVZW252GAV
s3://openneuro.org/ds005237/sub-NDARINVZX212UNE
s3://openneuro.org/ds005237/sub-NDARINVZY232VM1
s3://openneuro.org/ds005237/sub-NDARINVZY305TZ5
"""

# Extract the sub-IDs from the S3 paths
subject_ids = [line.split('/')[-1] for line in openneuro_subject_list.strip().split('\n')]

# Directory containing *_task-rest_desc-FCnetwork.csv files
fc_dir = 'derivatives/fc data'  # update this path to where the FC files are stored

# Path to your depression key
depression_key_path = 'depression_factors_binary_withoutmild.tsv'

# Load depression key and clean IDs
key_df = pd.read_csv(depression_key_path, sep='\t')
key_df['participant_id'] = key_df['participant_id'].str.strip()

rows = []



In [5]:
# Process only FC files that correspond to the valid subject IDs
for fp in glob.glob(os.path.join(fc_dir, '*_task-rest_desc-FCnetwork.csv')):
    basename = os.path.basename(fp)
    match = re.search(r'sub-[A-Za-z0-9]+', basename)
    if not match:
        print(f'Warning: could not extract ID from {basename}')
        continue
    subject_id = match.group(0)
    if subject_id not in subject_ids:
        continue  # skip files not in the OpenNeuro list

    fc_df = pd.read_csv(fp)
    fc_df['network_connection'] = fc_df['network_connection'].str.strip()
    fc_series = fc_df.set_index('network_connection')['fc']
    row = fc_series.to_frame().T
    row['participant_id'] = subject_id
    rows.append(row)

# Combine into a single wide table
fc_wide_df = pd.concat(rows, ignore_index=True)

# Merge with depression labels
merged_df = fc_wide_df.merge(key_df, on='participant_id', how='left')

# Remove rows without a depression_binary score
merged_df = merged_df.dropna(subset=['depression_binary'])

# Sanity check and save
assert merged_df['participant_id'].is_unique
merged_df.to_csv('fc_network_wide_without_mild_depression.tsv', sep='\t', index=False)


ValueError: No objects to concatenate